# 레이블링과 외곽선 검출
* 레이블링
* 외곽선 검출

## 레이블링의 이해
*  
* **cnt, labels = cv2.connectedComponents(src)** : 레이블 수행
* **cv2.connectedComponentsWithStats()** : + 통계 정보

## (실습) 레이블링

In [2]:
import cv2
import numpy as np

def labeling_basic():
    src = np.array([[0, 0, 1, 1, 0, 0, 0, 0],
                    [1, 1, 1, 1, 0, 0, 1, 0],
                    [1, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 1, 1, 0],
                    [0, 0, 0, 1, 1, 1, 1, 0],
                    [0, 0, 0, 1, 0, 0, 1, 0],
                    [0, 0, 1, 1, 1, 1, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0]]).astype(np.uint8)
    
    src = src * 255
    
    cnt, labels = cv2.connectedComponents(src)
    
    print('src :'),
    print(src)
    print('labels :')
    print(labels)
    print('number of labels :', cnt)
    
labeling_basic()

src :
[[  0   0 255 255   0   0   0   0]
 [255 255 255 255   0   0 255   0]
 [255 255 255 255   0   0   0   0]
 [  0   0   0   0   0 255 255   0]
 [  0   0   0 255 255 255 255   0]
 [  0   0   0 255   0   0 255   0]
 [  0   0 255 255 255 255 255   0]
 [  0   0   0   0   0   0   0   0]]
labels :
[[0 0 1 1 0 0 0 0]
 [1 1 1 1 0 0 2 0]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 0 3 3 0]
 [0 0 0 3 3 3 3 0]
 [0 0 0 3 0 0 3 0]
 [0 0 3 3 3 3 3 0]
 [0 0 0 0 0 0 0 0]]
number of labels : 4


## 레이블링 응용
* **cv2.connectedComponentsWithStats(src_bin)** : 레이블 수행 + 통계 정보

![](labeling.png)

## (실습) 레이블링 응용
키보드에서 흰색 글자만을 찾아서 노란색 사각형으로 표시

In [4]:
def labeling_stats():
    src = cv2.imread('keyboard.png', cv2.IMREAD_GRAYSCALE)
    
    _, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)
    
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    
    for i in range(1, cnt):
        (x, y, w, h, area) = stats[i]
        
        if area < 20 :
            continue
            
        pt1 = (x, y)
        pt2 = (x+w, y+h)
        cv2.rectangle(dst, pt1, pt2, (0, 255, 255))
        
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
labeling_stats()

![](labeling2.png)

<hr>

## 외곽선 검출
* (if, hole이 있는 객체의 경우) 안쪽 홀 외곽선과 객체 바깥쪽 외곽선으로 구분
* **contours, _ = cv2.findContours()** : 외곽선 검출 함수 (외곽선 정보 return)
    * mode : 검출 모드 (RETR_EXTERNAL, RETR_LIST, RETR_CCOMP, RETR_TREE)
    * method : 검출된 외곽선 점들의 좌표를 근사회하는 방법
* **cv2.drawContours()** : 검출한 외곽선 정보를 이용해 영상 위에 외곽선 그리기

## 외곽선 계층 구조
* 외곽선의 포함 관계에 의해 결정되는 구조 (부모 외곽선, 자식 외곽선 관계)

![](draw_c.png)

![](mode.png)

## (실습) 외곽선 검출 - RETR_LIST 검출모드
* 이진 영상에서 모든 외곽선을 찾아 각기 다른 색상으로 외곽선 그리기
* 모든 객체의 바깥쪽과 안쪽 홀 외곽선을 모두 검출
* 부모, 자식 간의 계층 정보는 X!

In [7]:
def contours_basic():
    src = cv2.imread('contours.png', cv2.IMREAD_GRAYSCALE)
    
    contours, _ = cv2.findContours(src, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    
    for i in range(len(contours)):
        c = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
        cv2.drawContours(dst, contours, i, c, 2)
        
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
contours_basic()

![](draw_contours.png)

## (실습) 외곽선 검출 - RETR_CCOMP 검출모드
2단계로 구성된 계층 구조 생성

In [8]:
def contours_hier():
    src = cv2.imread('contours.png', cv2.IMREAD_GRAYSCALE)
    
    contours, hierarchy = cv2.findContours(src, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    
    idx = 0
    while idx >= 0:
        c = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
        cv2.drawContours(dst, contours, idx, c, -1, cv2.LINE_8, hierarchy)
        idx = hierarchy[0, idx, 0]
        
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
contours_hier()

![](hier.png)

## (실습) 외곽선 처리 함수
* 입력 영상을 이진화하여 객체 영역 모두 검출
* 검출한 객체의 외곽선 정보를 이용해, 삼각형, 사각형, 원을 판단하여 화면에 표시 (객체의 외곽선 주변에 바운딩 박스 그리기)

In [13]:
import math

def setLabel(img, pts, label):
    (x, y, w, h) = cv2.boundingRect(pts)
    pt1 = (x, y)
    pt2 = (x+w, y+h)
    cv2.rectangle(img, pt1, pt2, (0, 0, 255), 1)
    cv2.putText(img, label, pt1, cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
    
def main():
    img = cv2.imread('polygons.jpg', cv2.IMREAD_COLOR)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img_bin = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for pts in contours:
        if cv2.contourArea(pts) < 400:
            continue
            
        approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)
        
        vtc = len(approx)
        
        if vtc == 3:
            setLabbel(img, pts, 'TRI')
        elif vtc == 4:
            setLabel(img, pts, 'RECT')
        else:
            lenth = cv2.arcLength(pts, True)
            area = cv2.contourArea(pts)
            ratio = 4. * math.pi * area / (lenth*lenth)
            
            if ratio > 0.85:
                setLabel(img, pts, 'CIR')
                
    cv2.imshow('img', img)
    cv2.waitKey()
    cv2.destroyAllWindows()
        
main()

![](polygons_result.png)